In [17]:
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import glob 
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
import random

In [2]:
images = []
images_path = glob.glob("plate_dataset/*.jpg")
labels = [0]

In [3]:
print(images_path[0])

plate_dataset/63.jpg


In [13]:
def yolobbox2bbox(x,y,w,h,dw,dh):
    x1, y1 = (x-w/2)*dw, (y-h/2)*dh
    x2, y2 = (x+w/2)*dw, (y+h/2)*dh
    return ((x1), (y1), (x2), (y2))

In [ ]:
aug_num = 0
for img_path in images_path:
    img = cv2.imread(img_path)
    images.append(img)
    img_name = img_path[:-3]
    label_path = img_name + 'txt'
    dh, dw, _ = img.shape
    with open(label_path,'r') as file:
        xys = []
        for line in file:
            label,x,y,w,h = [float(x) for x in line.split()]
            if(label in labels):
                x1,y1,x2,y2 = yolobbox2bbox(x,y,w,h,dw,dh)
                xys.append((x1,y1,x2,y2,label))
            else:
                print("UNKNOWN LABEL FOUND!")
                break
        bbs = BoundingBoxesOnImage([BoundingBox(x1,y1,x2,y2,label) for x1,y1,x2,y2,label in xys],shape=img.shape)
        aug_img, aug_bbs = augmentation(image = img,bounding_boxes=bbs)
        save_file(f"plate_dataset/augmented_plate_{aug_num}.jpg",aug_img)
        dh, dw, _ = aug_img.shape
        with open(f"plate_dataset/augmented_plate_{aug_num}.txt",'w') as f:
            for box in aug_bbs.bounding_boxes:
                x,y,w,h = pascal_voc_to_yolo(box.x1,box.y1,box.x2,box.y2,dw,dh)
                f.write(f"{int(label)} {x} {y} {w} {h}\n")
        aug_num+=1

In [ ]:
augmentation = iaa.Sequential([
        iaa.OneOf([ ## rotate
            iaa.Affine(rotate=0),
            iaa.Affine(rotate=90),
            iaa.Affine(rotate=180),
            iaa.Affine(rotate=270),
        ]),

        iaa.Fliplr(0.5),
        iaa.Flipud(0.2),

        iaa.OneOf([ # drop out augmentation
            iaa.CoarseDropout((0.0, 0.05), size_percent=(0.02, 0.25)),
            ]),

        iaa.OneOf([ ## weather augmentation
            iaa.Rain(speed=(0.3, 0.5)),
        ]),  

        iaa.OneOf([ ## brightness or contrast
            iaa.Multiply((0.8, 1.0)),
            iaa.contrast.LinearContrast((0.9, 1.1)),
        ]),

        iaa.OneOf([ ## blur or sharpen
            iaa.GaussianBlur(sigma=(0.0, 0.1)),
            iaa.Sharpen(alpha=(0.0, 0.1)),
        ])
    ],
    # do all of the above augmentations in random order
    random_order=True
)

In [14]:
def save_file(img_name,img):
    cv2.imwrite(img_name, img)
def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h) 
def pascal_voc_to_yolo(x1, y1, x2, y2, image_w, image_h):
    return [((x2 + x1)/(2*image_w)), ((y2 + y1)/(2*image_h)), (x2 - x1)/image_w, (y2 - y1)/image_h]

In [16]:
img_file = "plate_dataset/zoomed_augmented_plate_0.jpg"
label_path = "plate_dataset/zoomed_augmented_plate_0.txt"
image = cv2.imread(img_file)
dh, dw, _ = image.shape
with open(label_path,'r') as file:
    xys=[]
    for line in file:
        label,x,y,w,h = [float(x) for x in line.split()]
        print(label,x,y,w,h)
        if(label in labels):
            x1,y1,x2,y2 = yolobbox2bbox(x,y,w,h,dw,dh)
            print(x1,y1,x2,y2)
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 1)
            xys.append((x1,y1,x2,y2,label))
    bbs = BoundingBoxesOnImage([BoundingBox(x1,y1,x2,y2,label) for x1,y1,x2,y2,label in xys],shape=image.shape)
    cv2.imshow('image2',image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

0.0 0.438559 0.44719500000000006 0.11016899999999992 0.12871300000000005
180.999964 116.0000655 232.99973199999997 155.00010450000002


In [ ]:
img = images_path[0]
img = cv2.imread(img)
augmentation = iaa.Sequential([
        iaa.Affine(scale=0.5)
    ],
    # do all of the above augmentations in random order
    random_order=True
)
aug_img = augmentation(image = img)
cv2.imshow('image2',aug_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
augmentation = iaa.Sequential([
        iaa.Affine(scale=(0.2,0.5))
    ],
)

In [20]:
import cv2
import random

# Shuffle the list of image paths
random.shuffle(images_path)

# Select the first 3000 paths
selected_images_path = images_path[:3000]

aug_num = 0
for img_path in selected_images_path:
    img = cv2.imread(img_path)
    images.append(img)
    img_name = img_path[:-3]
    label_path = img_name + 'txt'
    dh, dw, _ = img.shape
    with open(label_path, 'r') as file:
        xys = []
        for line in file:
            label, x, y, w, h = [float(x) for x in line.split()]
            if label in labels:
                x1, y1, x2, y2 = yolobbox2bbox(x, y, w, h, dw, dh)
                xys.append((x1, y1, x2, y2, label))
            else:
                print("UNKNOWN LABEL FOUND!")
                break
        bbs = BoundingBoxesOnImage([BoundingBox(x1, y1, x2, y2, label) for x1, y1, x2, y2, label in xys], shape=img.shape)
        aug_img, aug_bbs = augmentation(image=img, bounding_boxes=bbs)
        save_file(f"plate_dataset/zoomed_augmented_plate_{aug_num}.jpg", aug_img)
        dh, dw, _ = aug_img.shape
        with open(f"plate_dataset/zoomed_augmented_plate_{aug_num}.txt", 'w') as f:
            for box in aug_bbs.bounding_boxes:
                x, y, w, h = pascal_voc_to_yolo(box.x1, box.y1, box.x2, box.y2, dw, dh)
                f.write(f"{int(label)} {x} {y} {w} {h}\n")
        aug_num += 1